In [79]:
# import basic library
import numpy as np
import pandas as pd
from PIL import Image

In [80]:
# connect google drive to load data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
# unzip data file and preprocess it
import os
import zipfile

!unzip '/content/HW4_data.zip'

#train data
X_train = []
for i in range(10000):
    X_train.append(np.asarray(Image.open("/content/HW4_data/public/train_{i}.jpg".format(i=i))))
    
X_train = np.array(X_train)
X_train.reshape(10000, 128, 128, 1)

df_train = pd.read_csv("/content/HW4_data/public/train_label.csv", index_col=0)
Y_train = pd.DataFrame(df_train, columns=['Edema'])
Y_train = np.array(Y_train)
Y_train_ad = pd.DataFrame(df_train, 
                          columns=['Atelectasis','Cardiomegaly','Edema','Lung Opacity','No Finding','Pleural Effusion','Support Devices'])
Y_train_ad = np.array(Y_train_ad)


#validation data
X_val = []
for i in range(1000):
    X_val.append(np.asarray(Image.open("/content/HW4_data/public/val_{i}.jpg".format(i=i))))
    
X_val = np.array(X_val)
X_val.reshape(1000, 128, 128, 1)


df_val = pd.read_csv("/content/HW4_data/public/val_label.csv", index_col=0)
Y_val = pd.DataFrame(df_val, columns=['Edema'])
Y_val = np.array(Y_val)
Y_val_ad = pd.DataFrame(df_val, 
                          columns=['Atelectasis','Cardiomegaly','Edema','Lung Opacity','No Finding','Pleural Effusion','Support Devices'])
Y_val_ad = np.array(Y_val_ad)

#test data
X_test = []
for i in range(1500):
    X_test.append(np.asarray(Image.open("/content/HW4_data/private/test_{i}.jpg".format(i=i))))

X_test = np.array(X_test)
X_test.reshape(1500, 128, 128, 1)


df_test_demo = pd.read_csv("/content/HW4_data/private/test_demo.csv")

串流輸出內容已截斷至最後 5000 行。
 extracting: HW4_data/public/train_6738.jpg  
 extracting: HW4_data/public/val_873.jpg  
 extracting: HW4_data/public/val_673.jpg  
 extracting: HW4_data/public/train_2361.jpg  
 extracting: HW4_data/public/train_1559.jpg  
 extracting: HW4_data/public/train_556.jpg  
 extracting: HW4_data/public/train_8835.jpg  
 extracting: HW4_data/public/train_8072.jpg  
 extracting: HW4_data/public/train_5806.jpg  
 extracting: HW4_data/public/train_336.jpg  
 extracting: HW4_data/public/train_4072.jpg  
 extracting: HW4_data/public/train_2778.jpg  
 extracting: HW4_data/public/train_1667.jpg  
 extracting: HW4_data/public/train_4722.jpg  
 extracting: HW4_data/public/train_13.jpg  
 extracting: HW4_data/public/train_6297.jpg  
 extracting: HW4_data/public/train_6576.jpg  
 extracting: HW4_data/public/train_2489.jpg  
 extracting: HW4_data/public/val_699.jpg  
 extracting: HW4_data/public/train_1589.jpg  
 extracting: HW4_data/public/train_4449.jpg  
 extracting: HW4_data/publ

In [82]:
#Basic Part

import tensorflow as tf
import keras



from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

#model structure

model = keras.models.Sequential()
model.add( Conv2D(32, kernel_size=(5, 5), activation='relu',input_shape=(128,128,1),strides=(3,3), padding='same') )
model.add( MaxPooling2D((3, 3),padding='same') )
model.add( Dropout(0.25) )
model.add( Conv2D(64, kernel_size=(3, 3), activation='relu',input_shape=(128,128,1),strides=(1,1),padding='same') )
model.add( MaxPooling2D((2, 2),padding='same') )
model.add( Dropout(0.25) )
model.add( Conv2D(128, kernel_size=(1, 1), activation='relu',input_shape=(128,128,1),strides=(1,1),padding='same') )
model.add( Dropout(0.25) )
model.add( Flatten() )
model.add( Dense(128, activation='relu') )
model.add( Dropout(0.5))
model.add( Dense(1, activation='sigmoid') )

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 43, 43, 32)        832       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 dropout_32 (Dropout)        (None, 15, 15, 32)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 15, 15, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 8, 8, 64)         0         
 g2D)                                                            
                                                                 
 dropout_33 (Dropout)        (None, 8, 8, 64)         

In [83]:
import sklearn
from sklearn.metrics import f1_score
# compile

model.compile( optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'] )

In [84]:
#train
tf.config.experimental_run_functions_eagerly(True)
history = model.fit( X_train, Y_train, epochs=10 ,verbose=1 ,validation_split=0.3 )


Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


219/219 [==============================] - 35s 162ms/step - loss: 1.1444 - accuracy: 0.6940 - val_loss: 0.5072 - val_accuracy: 0.6973
Epoch 2/10
219/219 [==============================] - 34s 153ms/step - loss: 0.5010 - accuracy: 0.7347 - val_loss: 0.4706 - val_accuracy: 0.7643
Epoch 3/10
219/219 [==============================] - 35s 159ms/step - loss: 0.4799 - accuracy: 0.7604 - val_loss: 0.4676 - val_accuracy: 0.7760
Epoch 4/10
219/219 [==============================] - 33s 152ms/step - loss: 0.4690 - accuracy: 0.7680 - val_loss: 0.4505 - val_accuracy: 0.7880
Epoch 5/10
219/219 [==============================] - 35s 158ms/step - loss: 0.4559 - accuracy: 0.7811 - val_loss: 0.4384 - val_accuracy: 0.8033
Epoch 6/10
219/219 [==============================] - 33s 151ms/step - loss: 0.4482 - accuracy: 0.7757 - val_loss: 0.4258 - val_accuracy: 0.8060
Epoch 7/10
219/219 [==============================] - 33s 151ms/step - loss: 0.4430 - accuracy: 0.7887 - val_loss: 0.4086 - val_accuracy: 0.8

In [85]:
#validation
validation = model.predict(X_val)
print(validation)
val_ans = np.where(validation>=0.5, 1, 0)
val_ans = np.array(val_ans)

score = f1_score(Y_val, val_ans, average='binary')

print(score)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


[[1.13763183e-01]
 [3.85069847e-02]
 [3.19240987e-02]
 [4.35499787e-01]
 [5.78113139e-01]
 [8.19298923e-02]
 [4.76653308e-01]
 [5.66085875e-01]
 [4.54963356e-01]
 [4.23586369e-03]
 [5.77836156e-01]
 [5.08815050e-02]
 [7.10952878e-02]
 [1.70549333e-01]
 [4.38311070e-01]
 [2.25616634e-01]
 [3.86141807e-01]
 [2.30018198e-02]
 [3.76085877e-01]
 [3.54504049e-01]
 [6.42792523e-01]
 [5.23099124e-01]
 [4.01073337e-01]
 [6.75147772e-03]
 [4.41068083e-01]
 [2.14025170e-01]
 [6.35967851e-02]
 [5.68729222e-01]
 [1.53099000e-01]
 [5.02362669e-01]
 [5.18620610e-02]
 [4.25792336e-01]
 [4.97484773e-01]
 [1.49938524e-01]
 [8.52679014e-02]
 [3.58011782e-01]
 [2.78873146e-02]
 [4.79254335e-01]
 [7.63108134e-02]
 [4.42873627e-01]
 [3.69067967e-01]
 [4.89572287e-01]
 [8.09490681e-03]
 [5.28824508e-01]
 [5.74325442e-01]
 [3.79372686e-01]
 [2.17375159e-02]
 [6.78973913e-01]
 [1.90024376e-02]
 [5.37598372e-01]
 [1.95993900e-01]
 [3.48606229e-01]
 [3.79035592e-01]
 [5.00020862e-01]
 [5.45793176e-02]
 [7.110998

In [86]:
#predict
pred = model.predict(X_test);
pred_ans = []
for i in pred:
  if i<0.5:
    pred_ans.append(0)
  else:
    pred_ans.append(1)
pred_ans = np.array(pred_ans)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [87]:
#Basic output

student_id = '108062213'
output_path = student_id + '_basic_prediction.csv'

basic_pred = pd.DataFrame( {'Edema':  pred_ans} )
basic_pred.to_csv(output_path, index=True)

In [109]:
#Advance

#model structure

model_ad = keras.models.Sequential()
model_ad.add( Conv2D(32, kernel_size=(5, 5), activation='relu',input_shape=(128,128,1),strides=(2,2), padding='same') )
model_ad.add( MaxPooling2D((3, 3),padding='same') )
model_ad.add( Dropout(0.25) )
model_ad.add( Conv2D(64, kernel_size=(3, 3), activation='relu',input_shape=(128,128,1),strides=(1,1),padding='same') )
model_ad.add( MaxPooling2D((2, 2),padding='same') )
model_ad.add( Dropout(0.25) )
model_ad.add( Conv2D(128, kernel_size=(1, 1), activation='relu',input_shape=(128,128,1),strides=(1,1),padding='same') )
model_ad.add( Dropout(0.25) )
model_ad.add( Flatten() )
model_ad.add( Dense(128, activation='relu') )
model_ad.add( Dropout(0.5))
model_ad.add( Dense(7, activation='sigmoid') )

model_ad.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 64, 64, 32)        832       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 22, 22, 32)       0         
 g2D)                                                            
                                                                 
 dropout_48 (Dropout)        (None, 22, 22, 32)        0         
                                                                 
 conv2d_37 (Conv2D)          (None, 22, 22, 64)        18496     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 11, 11, 64)       0         
 g2D)                                                            
                                                                 
 dropout_49 (Dropout)        (None, 11, 11, 64)      

In [110]:
# compile

model_ad.compile( optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'] )

In [111]:
#Train

history = model_ad.fit( X_train, Y_train_ad, epochs=20 ,verbose=1 ,validation_split=0.3 )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/20
219/219 [==============================] - 66s 300ms/step - loss: 0.7822 - accuracy: 0.6156 - val_loss: 0.4215 - val_accuracy: 0.6853
Epoch 2/20
219/219 [==============================] - 66s 301ms/step - loss: 0.4005 - accuracy: 0.6699 - val_loss: 0.3792 - val_accuracy: 0.7043
Epoch 3/20
219/219 [==============================] - 65s 299ms/step - loss: 0.3848 - accuracy: 0.6870 - val_loss: 0.3829 - val_accuracy: 0.6980
Epoch 4/20
219/219 [==============================] - 66s 300ms/step - loss: 0.3716 - accuracy: 0.6943 - val_loss: 0.3595 - val_accuracy: 0.7000
Epoch 5/20
219/219 [==============================] - 65s 299ms/step - loss: 0.3679 - accuracy: 0.6973 - val_loss: 0.3640 - val_accuracy: 0.6867
Epoch 6/20
219/219 [==============================] - 62s 282ms/step - loss: 0.3528 - accuracy: 0.6976 - val_loss: 0.3624 - val_accuracy: 0.6783
Epoch 7/20
219/219 [==============================] - 61s 281ms/step - loss: 0.3545 - accuracy: 0.6977 - val_loss: 0.3507 - val_ac

In [112]:
#validation
validation_ad = model_ad.predict(X_val, verbose=1)
print(validation_ad)
val_ad_ans = np.where( (validation_ad>=0.5) , 1 , 0)
val_ad_ans = np.array(val_ad_ans)

score_ad = f1_score(Y_val_ad, val_ad_ans, average='weighted')
print(score_ad)

 2/32 [>.............................] - ETA: 1s

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


32/32 [==============================] - 2s 68ms/step
[[8.5684359e-03 2.2893369e-02 1.1685446e-01 ... 8.8637233e-01
  2.0102739e-02 1.6310966e-01]
 [1.0038108e-02 2.3473293e-02 1.2699673e-01 ... 8.7344819e-01
  2.2763401e-02 1.3727054e-01]
 [9.5084310e-04 4.0704310e-03 2.5593132e-02 ... 9.7500443e-01
  2.3794770e-03 4.7702342e-02]
 ...
 [1.9245392e-01 2.8585649e-01 6.2991995e-01 ... 3.6783952e-01
  3.7700135e-01 3.8868338e-01]
 [1.9636869e-02 3.1094223e-02 1.6996324e-01 ... 8.3120930e-01
  3.9698213e-02 2.6439530e-01]
 [2.0088434e-01 2.8823477e-01 6.4632469e-01 ... 3.5185933e-01
  3.9117473e-01 4.0081483e-01]]
0.5087549687590339


In [113]:
#Advance predict
pred_ad = model_ad.predict(X_test);
print(pred_ad)

pred_ad_ans = np.where( (pred_ad>=0.5) , 1 , 0)
pred_ad_ans = np.array(pred_ad_ans)
print(pred_ad_ans)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


[[0.02941504 0.06323564 0.26723433 ... 0.73348147 0.07408556 0.28043282]
 [0.03627291 0.07335487 0.2755112  ... 0.72450894 0.07856092 0.2075488 ]
 [0.02024546 0.04980955 0.22841209 ... 0.7708278  0.05637985 0.17160198]
 ...
 [0.04824251 0.09061894 0.344853   ... 0.6544202  0.11132172 0.29804954]
 [0.01196042 0.03364894 0.14445657 ... 0.8550496  0.02862114 0.14098698]
 [0.18201482 0.26587415 0.6019375  ... 0.39598408 0.35479182 0.3885247 ]]
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 1 ... 0 0 0]]


In [114]:
#Advance output

output_path = student_id + '_advanced_prediction.csv'
advanced_pred = pd.DataFrame(pred_ad_ans)
advanced_pred.columns = ['Atelectasis','Cardiomegaly','Edema','Lung Opacity','No Finding','Pleural Effusion','Support Devices']
advanced_pred.to_csv(output_path, index=True)